---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    import re
    
    df = pd.read_csv("university_towns.txt", sep='\n', names=['RegionName'])
    
    # create & clean states
    states_mask = df['RegionName'].str.contains('\[ed')
    df['State'] = df.where(states_mask).fillna(method='ffill')
    df['State'] = df['State'].apply(lambda x: x.split('[ed')[0])
    
    # clean cities
    df = df.where(~states_mask).dropna().reset_index()
    df['RegionName'] = df['RegionName'].apply(lambda x: re.sub(r'\(.*', '', x).strip())
    
    # reorder columns
    df = df[['State', 'RegionName']]
    
    return df

get_list_of_university_towns().head()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo


In [4]:
# sophies test for cities, 
# from https://www.coursera.org/learn/python-data-analysis/discussions/weeks/4/threads/QRjk3bKzEee5Ew6CaivGLg

import re
import pandas as pd
import numpy as np
# list of unique states
stateStr = """
Ohio, Kentucky, American Samoa, Nevada, Wyoming
,National, Alabama, Maryland, Alaska, Utah
,Oregon, Montana, Illinois, Tennessee, District of Columbia
,Vermont, Idaho, Arkansas, Maine, Washington
,Hawaii, Wisconsin, Michigan, Indiana, New Jersey
,Arizona, Guam, Mississippi, Puerto Rico, North Carolina
,Texas, South Dakota, Northern Mariana Islands, Iowa, Missouri
,Connecticut, West Virginia, South Carolina, Louisiana, Kansas
,New York, Nebraska, Oklahoma, Florida, California
,Colorado, Pennsylvania, Delaware, New Mexico, Rhode Island
,Minnesota, Virgin Islands, New Hampshire, Massachusetts, Georgia
,North Dakota, Virginia
"""
#list of regionName entries string length
regNmLenStr = """
06,08,12,10,10,04,10,08,09,09,05,06,11,06,12,09,08,10,12,06,
06,06,08,05,09,06,05,06,10,28,06,06,09,06,08,09,10,35,09,15,
13,10,07,21,08,07,07,07,12,06,14,07,08,16,09,10,11,09,10,06,
11,05,06,09,10,12,06,06,11,07,08,13,07,11,05,06,06,07,10,08,
11,08,13,12,06,04,08,10,08,07,12,05,06,09,07,10,16,10,06,12,
08,07,06,06,06,11,14,11,07,06,06,12,08,10,11,06,10,14,04,11,
18,07,07,08,09,06,13,11,12,10,07,12,07,04,08,09,09,13,08,10,
16,09,10,08,06,08,12,07,11,09,07,09,06,12,06,09,07,10,09,10,
09,06,15,05,10,09,11,12,10,10,09,13,06,09,11,06,11,09,13,37,
06,13,06,09,49,07,11,12,09,11,11,07,12,10,06,06,09,04,09,15,
10,12,05,09,08,09,09,07,14,06,07,16,12,09,07,09,06,32,07,08,
08,06,10,36,09,10,09,06,09,11,09,06,10,07,14,08,07,06,10,09,
05,11,07,06,08,07,05,07,07,04,06,05,09,04,25,06,07,08,05,08,
06,05,11,09,07,07,06,13,09,05,16,05,10,09,08,11,06,06,06,10,
09,07,06,07,10,05,08,07,06,08,06,30,09,07,06,11,07,12,08,09,
16,12,11,08,06,04,10,10,15,05,11,11,09,08,06,04,10,10,07,09,
11,08,26,07,13,05,11,03,08,07,06,05,08,13,10,08,08,08,07,07,
09,05,04,11,11,07,06,10,11,03,04,06,06,08,08,06,10,09,05,11,
07,09,06,12,13,09,10,11,08,07,07,08,09,10,08,10,08,56,07,12,
07,16,08,04,10,10,10,10,07,09,08,09,09,10,07,09,09,09,12,14,
10,29,19,07,11,12,13,13,09,10,12,12,12,08,10,07,10,07,07,08,
08,08,09,10,09,11,09,07,09,10,11,11,10,09,09,12,09,06,08,07,
12,09,07,07,06,06,08,06,15,08,06,06,10,10,10,07,05,10,07,11,
09,12,10,12,04,10,05,05,04,14,07,10,09,07,11,10,10,10,11,15,
09,14,12,09,09,07,12,04,10,10,06,10,07,28,06,10,08,09,10,10,
10,13,12,08,10,09,09,07,09,09,07,11,11,13,08,10,07
"""

df = get_list_of_university_towns()

cols = ["State", "RegionName"]

print('Shape test: ', "Passed" if df.shape ==
      (517, 2) else 'Failed')
print('Index test: ',
      "Passed" if df.index.tolist() == list(range(517))
      else 'Failed')

print('Column test: ',
      "Passed" if df.columns.tolist() == cols else 'Failed')
print('\\n test: ',
      "Failed" if any(df[cols[0]].str.contains(
          '\n')) or any(df[cols[1]].str.contains('\n'))
      else 'Passed')
print('Trailing whitespace test:',
      "Failed" if any(df[cols[0]].str.contains(
          '\s+$')) or any(df[cols[1]].str.contains(
              '\s+$'))
      else 'Passed')
print('"(" test:',
      "Failed" if any(df[cols[0]].str.contains(
          '\(')) or any(df[cols[1]].str.contains(
              '\('))
      else 'Passed')
print('"[" test:',
      "Failed" if any(df[cols[0]].str.contains(
          '\[')) or any(df[cols[1]].str.contains(
              '\]'))
      else 'Passed')

states_vlist = [st.strip() for st in stateStr.split(',')]

mismatchedStates = df[~df['State'].isin(
    states_vlist)].loc[:, 'State'].unique()
print('State test: ', "Passed" if len(
    mismatchedStates) == 0 else "Failed")
if len(mismatchedStates) > 0:
    print()
    print('The following states failed the equality test:')
    print()
    print('\n'.join(mismatchedStates))

df['expected_length'] = [int(s.strip())
                         for s in regNmLenStr.split(',')
                         if s.strip().isdigit()]
regDiff = df[df['RegionName'].str.len() != df['expected_length']].loc[
    :, ['RegionName', 'expected_length']]
regDiff['actual_length'] = regDiff['RegionName'].str.len()
print('RegionName test: ', "Passed" if len(regDiff) ==
      0 else ' \nMismatching regionNames\n {}'.format(regDiff))

Shape test:  Passed
Index test:  Passed
Column test:  Passed
\n test:  Passed
Trailing whitespace test: Passed
"(" test: Passed
"[" test: Passed
State test:  Passed
RegionName test:  Passed


In [5]:
def parse_gdplev():
    df = pd.read_excel("gdplev.xls", skiprows=8, usecols=[4,6], header=None, names=['Quarters', 'GDP'])
    df = (df.where(df['Quarters'].str.startswith('2'))
          .dropna()
          .reset_index(drop=True)
         )
    
    return df

def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3
    
    A recession is defined as starting with two consecutive quarters 
    of GDP decline, and ending with two consecutive quarters of GDP growth.
    '''
    
    df = parse_gdplev()
    
    for i in range(1, len(df) - 1):
        x = df.iloc[i]
        if df.at[i-1, 'GDP'] > x['GDP'] and x['GDP'] > df.at[i+1, 'GDP']:
            return x['Quarters']
    
    return None
    
get_recession_start()

'2008q3'

In [6]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3
    
    A recession is defined as starting with two consecutive quarters 
    of GDP decline, and ending with two consecutive quarters of GDP growth.
    '''
    
    df = parse_gdplev()
    start = df[df['Quarters'] == get_recession_start()].first_valid_index()

    for i in range(start, len(df) - 2):
        x = df.iloc[i]
        if x['GDP'] < df.at[i+1, 'GDP'] and df.at[i+1, 'GDP'] < df.at[i+2, 'GDP']:
            return df.at[i+2, 'Quarters']
       
    return None

get_recession_end()

'2009q4'

In [7]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3
    
    A recession bottom is the quarter within a recession which had the lowest GDP.
    '''
    
    df = parse_gdplev()

    recession = df[(df['Quarters'] >= get_recession_start()) & (df['Quarters'] <= get_recession_end())]
    
    return recession['Quarters'].loc[recession['GDP'].argmin()]

get_recession_bottom()

'2009q2'

In [17]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Q1 := jan(01) - march(03)
    Q2 := april(04) - june(06)
    Q3 := july(07) - sept(09)
    Q4 := oct(10) - dec(12)
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    
    From the Zillow research data site there is housing data for the United States. 
    In particular the datafile for all homes at a city level, City_Zhvi_AllHomes.csv, 
    has median home sale prices at a fine grained level.
    '''
    dates_to_keep = ["{0}-{1:02d}".format(year, month) for year in range(2000,2017) for month in range(1,13)][:-4]
    df = pd.read_csv("City_Zhvi_AllHomes.csv", usecols=['RegionName', 'State'] + dates_to_keep)
    df['State'] = df['State'].apply(lambda x: states.get(x))
    df = df.sort_values('State').set_index(['State', 'RegionName'])
    
    # create quarters
    qranges = {
        1 : ("01","03"),
        2 : ("04","06"),
        3 : ("07","09"),
        4 : ("10","12")
    }
    for year in range(2000, 2017):
        for quarter in range(1,5):
            qstart, qend = qranges.get(quarter)
            start = "{0}-{1}".format(year, qstart)
            end = "{0}-{1}".format(year, qend)
            colname = "{0}q{1}".format(year, quarter)
            # missing 2016-09
            if year == 2016 and quarter == 3:
                df[colname] = df.loc[:, start:"2016-08"].mean(axis=1)
                break
            else:
                df[colname] = df.loc[:, start:end].mean(axis=1)
    
    # filter unwanted quarters
    df = df.loc[:, "2000q1":"2016q3"]
    assert df.shape == (10730,67)
    
    return df
              
convert_housing_data_to_quarters().head()

2000q1         2000q2         2000q3  \
State   RegionName                                                 
Alabama Mulga                  NaN            NaN            NaN   
        Trussville   164400.000000  165900.000000  167600.000000   
        Satsuma       88600.000000   90966.666667   90333.333333   
        Irvington     72466.666667   74100.000000   75700.000000   
        Mount Olive  106500.000000  107333.333333  106966.666667   

                            2000q4         2001q1         2001q2  \
State   RegionName                                                 
Alabama Mulga                  NaN            NaN            NaN   
        Trussville   169766.666667  172533.333333  173433.333333   
        Satsuma       89600.000000   90233.333333   90766.666667   
        Irvington     76266.666667   76300.000000   74300.000000   
        Mount Olive  107200.000000  108266.666667  108766.666667   

                            2001q3         2001q4         2002q1  \
State   RegionName                                                 
Alabama Mulga                  NaN            NaN            NaN   
        Trussville   174166.666667  176700.000000  178000.000000   
        Satsuma       90633.333333   92200.000000   93000.000000   
        Irvington     73500.000000   73833.333333   75966.666667   
        Mount Olive  110166.666667  112400.000000  113966.666667   

                            2002q2    ...            2014q2         2014q3  \
State   RegionName                    ...                                    
Alabama Mulga                  NaN    ...      45466.666667   43600.000000   
        Trussville   180800.000000    ...     212033.333333  210966.666667   
        Satsuma       93200.000000    ...     125266.666667  127500.000000   
        Irvington     79266.666667    ...      90533.333333   95666.666667   
        Mount Olive  115000.000000    ...     132066.666667  132366.666667   

                            2014q4         2015q1         2015q2  \
State   RegionName                                                 
Alabama Mulga         42166.666667   41933.333333   42733.333333   
        Trussville   214566.666667  216600.000000  220066.666667   
        Satsuma      128266.666667  129266.666667  131300.000000   
        Irvington     98466.666667   99500.000000   98800.000000   
        Mount Olive  132066.666667  135433.333333  138100.000000   

                            2015q3         2015q4         2016q1  \
State   RegionName                                                 
Alabama Mulga         42033.333333   43033.333333   40666.666667   
        Trussville   219833.333333  219433.333333  220266.666667   
        Satsuma      133333.333333  134800.000000  134666.666667   
        Irvington     97600.000000   98000.000000  101500.000000   
        Mount Olive  138433.333333  140633.333333  142866.666667   

                            2016q2    2016q3  
State   RegionName                            
Alabama Mulga         38366.666667   38300.0  
        Trussville   223566.666667  225600.0  
        Satsuma      133533.333333  133100.0  
        Irvington    103966.666667  103950.0  
        Mount Olive  144033.333333  143850.0  

[5 rows x 67 columns]

In [19]:
import scipy.stats as stats

def get_quarter_b4_recession(qstart):
    year, quarter = int(qstart[:4]), int(qstart[-1])
    if quarter == 1:
        return "{0}q{1}".format(year-1, 4)
    else:
        return "{0}q{1}".format(year, quarter-1)

    
def run_ttest():
    '''Hypothesis: University towns have their mean housing prices less effected by recessions. 
    Run a t-test to compare the ratio of the mean price of houses in university towns the 
    quarter before the recession starts compared to the recession bottom. 
    (price_ratio=quarter_before_recession/recession_bottom)
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    # price_ratio=quarter_before_recession/recession_bottom
    df = convert_housing_data_to_quarters()
    df["ttest"] = df[get_quarter_b4_recession(get_recession_start())] / df[get_recession_bottom()]
    df = df["ttest"]
    
    # split uni & others
    unitowns = get_list_of_university_towns().set_index(['State', 'RegionName']).index.get_values()
    df_uni = df.loc[unitowns]
    df_others = df.drop(unitowns)
#     assert len(df) == len(df_uni) + len(df_others)
    
    # ttest
    s, pvalue = stats.ttest_ind(df_uni, df_others, nan_policy='omit')
    better = "university town" if df_uni.mean() < df_others.mean() else "non-university town"
    
    return (pvalue < 0.01, pvalue, better)

run_ttest()

(True, 0.0027240637047531249, 'university town')